In [1]:
import requests
import os
import shutil
import json
import pandas as pd
import yaml

import utils, request_models

import warnings
warnings.filterwarnings('ignore')


In [2]:
######## File paths ########
with open("../configs/self-driving-demo.yaml", "r") as f:
    config = yaml.safe_load(f)

protein_file_path = config['paths']['protein_file_path']
diffdock_output_dir = config['paths']['diffdock_output_dir']
dsmbind_input_dir = config['paths']['diffdock_output_dir']

starting_molecule_csv = config['paths']['starting_molecule_csv']
molmim_generated_csv = config['paths']['molmim_generated_csv']
dsmbind_predictions_csv = config['paths']['dsmbind_predictions_csv']

utils.prepare_output_directory(diffdock_output_dir)
utils.prepare_output_directory(dsmbind_input_dir)

utils.delete_datasets(molmim_generated_csv)
utils.delete_datasets(dsmbind_predictions_csv)

# Get folded protein
folded_protein = utils.file_to_json_compatible_string(protein_file_path)

# Get starting molecules
df_starting_molecules = pd.read_csv(starting_molecule_csv)

# Round 0
print("Round 0")
molecule_name = df_starting_molecules['Molecules'][0]
molecule = df_starting_molecules['Smiles'][0]

# Molecular Generation with MolMIM
molmim_response = request_models.call_molmim(molecule)
generated_ligands = '\n'.join([v['smiles'] for v in molmim_response['generated']])
utils.update_dataframe_molmim_generated_molecules(molmim_response['generated'], molecule_name)

# Protein-Ligand Docking with DiffDock
diffdock_response = request_models.call_diffdock(folded_protein, generated_ligands)
utils.create_diffdock_outputs_dsmbind_inputs(molecule_name, diffdock_response)

# Binding Affinity with DSMBind
os.system("python /workspace/bionemo/examples/molecule/dsmbind/infer.py")


Round 0
The CSV file exists.


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
[NeMo W 2024-12-18 18:47:13 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:257: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-12-18 18:47:13 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:268: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-12-18 18:47:13 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:328: FutureWarni

[NeMo I 2024-12-18 18:47:20 megatron_hiddens:110] Registered hidden transform sampled_var_cond_gaussian at bionemo.model.core.hiddens_support.SampledVarGaussianHiddenTransform
[NeMo I 2024-12-18 18:47:20 megatron_hiddens:110] Registered hidden transform interp_var_cond_gaussian at bionemo.model.core.hiddens_support.InterpVarGaussianHiddenTransform


[NeMo W 2024-12-18 18:47:20 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    


[NeMo I 2024-12-18 18:47:20 infer:68] 
    
    ************** Experiment Configuration ***********
[NeMo I 2024-12-18 18:47:20 infer:69] 
    data:
      processed: false
      raw_inference_data_dir: ${oc.env:BIONEMO_HOME}/generative-virtual-screening/data/dsmbind_inputs
      predictions_csv: ${oc.env:BIONEMO_HOME}/generative-virtual-screening/data/dsmbind_predictions.csv
      processed_inference_data_path: ${oc.env:BIONEMO_HOME}/examples/tests/test_data/molecule/dsmbind/inference/processed.pkl
    model:
      hidden_size: 256
      mpn_depth: 3
      num_heads: 4
      dropout: 0.1
      threshold: 10.0
      vocab_size: 38
      aa_size: 21
      max_residue_atoms: 14
    inference:
      num_gpus: 1
      ckpt_path: ${oc.env:BIONEMO_HOME}/models/molecule/dsmbind/dsmbind.pth
    
[NeMo I 2024-12-18 18:47:20 infer:75] ************** Load Trained Model ***********
[NeMo I 2024-12-18 18:47:21 infer:82] # Model parameters: 1019136
[NeMo I 2024-12-18 18:47:21 infer:94] **************

  0%|          | 0/10 [00:00<?, ?it/s][NeMo W 2024-12-18 18:47:21 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2332
      warnings.warn(
    
Failed to patch pandas - unable to change molecule rendering
[NeMo W 2024-12-18 18:47:21 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2332
      warnings.warn(
    
Failed to patch pandas - unable to change molecule rendering
[NeMo W 2024-12-18 18:47:21 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2332
      warnings.warn(
    
Failed to patch pandas - unable to change molecule rendering
[NeMo W 2024-12-18 18:47:21 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unre

'Molecule'
[NeMo I 2024-12-18 18:47:21 infer:96] ************** Loading Inference Dataset ***********
[NeMo I 2024-12-18 18:47:21 infer:104] ************** Starting Inference ***********
[NeMo I 2024-12-18 18:47:22 infer:106] Predictions: [-1626685.75, -1473968.25, -1604278.75, -1486546.375, -1616566.5, -1357558.125, -1624152.0, -1571350.5]


0

In [3]:

df_molmim = pd.read_csv(molmim_generated_csv) 
df_dsmbind = pd.read_csv(dsmbind_predictions_csv)
df_joined = pd.concat([df_molmim, df_dsmbind], axis=1)

df_joined.to_csv('../data/results.csv')

In [4]:
def update_dataframe_molmim_generated_molecules(molmim_generated, 
                                                starting_molecule_name, 
                                                molmim_generated_csv = molmim_generated_csv):
    import pandas as pd

    df = pd.DataFrame(molmim_generated)
    # Reset the index and make it a column
    df.reset_index(inplace=True)
    df.rename(columns={'smiles':'generated_smiles',
                    'score':'molmim_qed_score',
                    'index':'generated_compound_index'},
                    inplace=True)
    df['starting_molecule'] = starting_molecule_name

    if os.path.exists(molmim_generated_csv):
        print("The CSV file exists.")
        df_old = pd.read_csv(molmim_generated_csv)
        df_merged = pd.concat([df_old, df], ignore_index=True)

        df_merged.to_csv(molmim_generated_csv, index=False)
    else:
        df.to_csv(molmim_generated_csv, index=False)

In [5]:
rd = 0
while len(df_joined[df_joined['DSMBind_predictions'] < -1600000]) < 10:
    rd = rd + 1
    print(f"Round {rd}")

    molecule_name = df_starting_molecules['Molecules'][rd]
    molecule = df_starting_molecules['Smiles'][rd]

    # Molecular Generation with MolMIM
    print("MolMIM...")
    molmim_response = request_models.call_molmim(molecule)
    generated_ligands = '\n'.join([v['smiles'] for v in molmim_response['generated']])
    update_dataframe_molmim_generated_molecules(molmim_response['generated'], molecule_name)

    # Protein-Ligand Docking with DiffDock
    print("DiffDock...")
    diffdock_response = request_models.call_diffdock(folded_protein, generated_ligands)
    utils.create_diffdock_outputs_dsmbind_inputs(molecule_name, diffdock_response)

    # Binding Affinity with DSMBind
    print("DSMBind...")
    os.system("python /workspace/bionemo/examples/molecule/dsmbind/infer.py")

    df_molmim = pd.read_csv(molmim_generated_csv) 
    df_dsmbind = pd.read_csv(dsmbind_predictions_csv)
    df_joined = pd.concat([df_molmim, df_dsmbind], axis=1)

    df_joined.to_csv('../data/results.csv')


Round 1
MolMIM...
The CSV file exists.
DiffDock...
DSMBind...


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
[NeMo W 2024-12-18 18:48:42 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:257: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-12-18 18:48:42 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:268: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-12-18 18:48:42 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:328: FutureWarni

[NeMo I 2024-12-18 18:48:50 megatron_hiddens:110] Registered hidden transform sampled_var_cond_gaussian at bionemo.model.core.hiddens_support.SampledVarGaussianHiddenTransform
[NeMo I 2024-12-18 18:48:50 megatron_hiddens:110] Registered hidden transform interp_var_cond_gaussian at bionemo.model.core.hiddens_support.InterpVarGaussianHiddenTransform


[NeMo W 2024-12-18 18:48:50 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    


[NeMo I 2024-12-18 18:48:50 infer:68] 
    
    ************** Experiment Configuration ***********
[NeMo I 2024-12-18 18:48:50 infer:69] 
    data:
      processed: false
      raw_inference_data_dir: ${oc.env:BIONEMO_HOME}/generative-virtual-screening/data/dsmbind_inputs
      predictions_csv: ${oc.env:BIONEMO_HOME}/generative-virtual-screening/data/dsmbind_predictions.csv
      processed_inference_data_path: ${oc.env:BIONEMO_HOME}/examples/tests/test_data/molecule/dsmbind/inference/processed.pkl
    model:
      hidden_size: 256
      mpn_depth: 3
      num_heads: 4
      dropout: 0.1
      threshold: 10.0
      vocab_size: 38
      aa_size: 21
      max_residue_atoms: 14
    inference:
      num_gpus: 1
      ckpt_path: ${oc.env:BIONEMO_HOME}/models/molecule/dsmbind/dsmbind.pth
    
[NeMo I 2024-12-18 18:48:50 infer:75] ************** Load Trained Model ***********
[NeMo I 2024-12-18 18:48:51 infer:82] # Model parameters: 1019136
[NeMo I 2024-12-18 18:48:51 infer:94] **************

...

[NeMo W 2024-12-18 18:50:29 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    


[NeMo I 2024-12-18 18:50:29 infer:68] 
    
    ************** Experiment Configuration ***********
[NeMo I 2024-12-18 18:50:29 infer:69] 
    data:
      processed: false
      raw_inference_data_dir: ${oc.env:BIONEMO_HOME}/generative-virtual-screening/data/dsmbind_inputs
      predictions_csv: ${oc.env:BIONEMO_HOME}/generative-virtual-screening/data/dsmbind_predictions.csv
      processed_inference_data_path: ${oc.env:BIONEMO_HOME}/examples/tests/test_data/molecule/dsmbind/inference/processed.pkl
    model:
      hidden_size: 256
      mpn_depth: 3
      num_heads: 4
      dropout: 0.1
      threshold: 10.0
      vocab_size: 38
      aa_size: 21
      max_residue_atoms: 14
    inference:
      num_gpus: 1
      ckpt_path: ${oc.env:BIONEMO_HOME}/models/molecule/dsmbind/dsmbind.pth
    
[NeMo I 2024-12-18 18:50:29 infer:75] ************** Load Trained Model ***********
[NeMo I 2024-12-18 18:50:29 infer:82] # Model parameters: 1019136
[NeMo I 2024-12-18 18:50:29 infer:94] **************

  0%|          | 0/10 [00:00<?, ?it/s][NeMo W 2024-12-18 18:50:29 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2332
      warnings.warn(
    
Failed to patch pandas - unable to change molecule rendering
[NeMo W 2024-12-18 18:50:29 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2332
      warnings.warn(
    
Failed to patch pandas - unable to change molecule rendering
[NeMo W 2024-12-18 18:50:29 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2332
      warnings.warn(
    
Failed to patch pandas - unable to change molecule rendering
[NeMo W 2024-12-18 18:50:29 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unre

'Molecule'
[NeMo I 2024-12-18 18:50:29 infer:96] ************** Loading Inference Dataset ***********
[NeMo I 2024-12-18 18:50:29 infer:104] ************** Starting Inference ***********
[NeMo I 2024-12-18 18:50:30 infer:106] Predictions: [-1626685.75, -1473968.25, -1604278.75, -1486546.375, -1616566.5, -1357558.125, -1624152.0, -1571350.5]
Round 4


KeyError: 4

In [6]:
len(df_joined[df_joined['DSMBind_predictions'] < -1600000])

4

In [12]:
df_joined

,generated_compound_index,generated_smiles,molmim_qed_score,starting_molecule,DSMBind_predictions
0,0,CC(C)(C)[C@@H](NC(=O)C(F)(F)c1ccccc1)c1nc(C2CC...,0.865846,Nirmatrelvir,-1626685.750
1,1,CC(C)(C)[C@@H](NC(=O)C(F)(F)F)C(=O)N1CC2CCC1CC2,0.849637,Nirmatrelvir,-1473968.250
2,2,CC(C)(C)[C@@H](NC(=O)C(F)(F)F)C(=O)N1CC2CCC1CC2,0.849637,Nirmatrelvir,-1604278.750
3,3,CC(C)(C)[C@@H](NC(=O)C(F)(F)F)C(=O)N1CC2CCC1CC2,0.849637,Nirmatrelvir,-1486546.375
4,4,CC(C)(C)C(NC(=O)C(F)(F)F)C(=O)N1CC2CCC1CC2,0.849637,Nirmatrelvir,-1616566.500
5,0,Cn1cnc(CN2C[C@H]3C[C@@H](NC(=O)c4ccccc4)C[C@H]...,0.927308,Ensitrelvir,-1357558.125
6,1,Cn1cnc(CN2C[C@H]3C[C@@H](NC(=O)c4ccco4)C[C@H]2...,0.917014,Ensitrelvir,-1624152.000
7,2,Cn1cnc(CN2CC[C@]3(CNC(=O)OC(C)(C)C)CCC[C@@H]23)n1,0.911968,Ensitrelvir,-1571350.500
8,3,Cn1cnc(CN2CC[C@]3(CNC(=O)c4ccccc4)CCC[C@@H]23)n1,0.904817,Ensitrelvir,NaN
9,4,Cn1cnc(CN2CC[C@]3(CNC(=O)c4ccco4)CCC[C@@H]23)n1,0.901211,Ensitrelvir,NaN
